# Scalar, Vector, Matrix, Tensor

scalar - 1차원의 숫자
vector - 2차원의 숫자
matrix - vector들을 모아 만드는 2차원의 숫자 모음
tensor - vector, matrix들 모음

그림 데이터 같은 경우 3차원으로 된 숫자 데이터들이 순서대로 나열되어있는 데이터
그림 데이터를 Tensor로 처리


# SIMD 에서 Splat, Join, Add, Mul 함수

scalar 단위로 계산하는 것보다 훨씬 더 효율적

In [ ]:
alias dtype = DType.float32

let a = SIMD[dtype].splat(0.5)
let b = SIMD[dtype].splat(1.5)
# splat은 vector 혹은 tensor의 모든 원소에 같은 값을 집어넣도록 해주는 함수
# 새로운 벡터를 만드는데 아주 유용하게 쓰임

print("SIMD a: ", a)
print("SIMD b: ", b)
print()
print("SIMD JOIN: ", a.join(b))
# join은 두 벡터를 하나의 벡터로 만들어줌 > 차원이 증가
print("SIMD add: ", a.__add__(b))
# add는 두 벡터의 각 요소들을 순서대로 더해줌
print("SIMD mul: ", a.__mul__(b))
# mul은 두 벡터의 각 요소들을 순서대로 곱해줌


# Vectorize, Parallelize
평균을 빠르게 구하기 위한 벡터화, 병렬화

In [ ]:
from random import rand

from algorithm import vectorize, parallelize
from sys.info import simdwidthof
import benchmark


alias dtype = DType.float32
alias simd_width = simdwidthof[dtype]()

fn row_mean_naive[dtype: DType](t: Tensor[dtype]) -> Tensor[dtype]:
  var res = Tensor[dtype](t.dim(0), 1)

  for i in range(t.dim(0)):
    for j in range(t.dim(1)):
      res[i] += t[i, j]
    res[i] /=t.dim(1)

  return res

fn row_mean_updated[dtype: DType](t: Tensor[dtype]) -> Tensor[dtype]:
  var res = Tensor[dtype](t.dim(0), 1)
  @parameter
  fn parallel_reduce_rows(idx:Int) -> None:

    @parameter
    fn vectorize_reduce_row[simd_width: Int](idx2: Int) -> None:
      res[idx1] += t.simd_load[simd_width](idx1* t.dim(0) + idx2).reduce_add()

    vectorize[2*simd_width, vectorize_reduce_row](t.dim(1))
    res[idx1] /= t.dim(1)
  parallelize[parallel_reduce_rows](t.dim(0), t.dim(0))
  return res





fn main():
  let t = rand[dtype](1000, 1000000)
  var result = Tensor[dtype](t.dim(0),1)
  # _ = row_mean_naive(t)

  @parameter
  fn bech_mean():
    _ = row_mean_naive(t)

  @parameterfn
  fn bech_mean_updated():
    _ = row_mean_updated(t)

  let report = benchmark.run[bech_mean]()
  let report_updated = benchmark.run[bech_mean_updated]()
  report.print()
  report_updated.print()
  print("Speed up", report.mean()/report_updated.mean())


main()

# Square matrices 만들기, 초기화, 접근, 출력

정사각형의 행렬을 만들고, 초기화하고, 요소에 접근하고, 출력하는 효율적인 방식


In [ ]:
from algorithm import vectorize

struct SquateMatrix[dtype:DType = DType.float32, dim:Int = 3]():
  var mat: Tensor[dtype]

  fn __init__(inout self, value: SIMD[dtype, 1] = 6):
    self.mat = Tensor[dtype](self.dim, self.dim)

    @parameter
    fn fill_value[simd_width: Int](idx:Int) -> None:
      self.mat.simd_store(idx, self.mat.simd_load[simd_width](idx).splat(value))

    vectorize[simd_width, fill_value](self.mat.num_elementes())

  fn __getitem__(self, x: Int, y: Int) -> SIMD[dtype, 1]:
    return self.mat[x, y]

  fn print(self):
    print(self.mat)

SquateMatrix().print()
SquateMatrix(value=7).print()
SquateMatrix[DType.float64](value=9).print()
SquateMatrix[DType.float64,dim=5](value=9).print()

# Square matrices에 scalar 값 곱하기

정사각형의 행렬의 각 요소에 scalar 값 곱하기


In [ ]:
from sys.info import simdwidthof
from math import mul

fn multiply(sm: SquateMatrix, val: SIMD[sm.dtype, 1]) -> Tensor[sm.dtype]:
  alias simd_width: Int = simdwidthof[sm.dtype]()
  let result_tensor = Tensor[sm.dtype](sm.mat.shape())

  @parameter
  fn vectorize_multipy[simd_width:Int](idx: Int) -> None:
    result_tensor.simd_store[simd_width](idx, mul[sm.dtype, simd_width](sm.mat.simd_load[simd_width](idx), val))

  vectorize[simd_width, vectorize_multipy](sm.mat.num_elementes())
  return result_tensor


let sm = SquateMatrix(6)
sm.print()
let res = multipy(sm, 100.0)
print(res)